# Faire correspondre un modèle mathématique au code d'un solveur

## Introduction.
Dans ce notebook, on va analyser les méthodes pour faire correspondre un modèle mathématique donné à un code de solveur Python.
Autrement dit, comment passer de l'un à l'autre.
Cela dans le cadre d'optimisations combinatoires.

*** CREATION EN COURS ***

<b>Notebook réalisé par GITHUB ESTELLE DERRIEN</b>


<div style="text-align:center">
<img src="img/intro_correspondre.png">
</div>

## Sommaire

1. Les symboles mathématique à connaitre.
2. Notre exemple de base
3. Extention de notre exemple et symbole de sommation.
4. Entrer un symbole de sommation dans un solveur Python.
    - Exemple avec PULP.
    - Exemple avec CPLEX.
    - Exemple avec GLPK en ligne de commande.
6. Se familiariser avec l'écriture aij avec un exemple simple tiré de Hillier
    - 6.a Présentation du problème dans un tableau
    - 6.b Modélisation mathématique
    - 6.c Solution avec Python Pulp
7. Se familiariser avec une fonction objectif comprenant une sommation et une soustraction de coûts.
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
8. Se familiariser avec l'écriture double sommation ΣΣ
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
9. Modélisation d'une optimisation combinatoire plus complexe de type Chimie/Pétrole 
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
10. Modélisation d'un problème de mélange
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
11. Modélisation d'un problème de voyageur de commerce
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python 
12. Linéarisation d'un problème par breakpoints et modélisation.
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python 
13. Aborder une optimisation non linéaire



# 1. Les symboles mathématique à connaitre :

- Le symbole Σ (sigma), il s'apprends en classe de seconde et permet de simplifier l'ecriture d'une addition.

- Le symbole ∈ veut dire "appartient à", on s'en sert quand on a au préalable définit un ensemble S{x1,x2...xn}

- Le symbole ∀ veut dire "Pour tout"

- Une matrice Aij : i c'est la ligne, j c'est la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4.

- Une fonction objectif : elle est destinée à être minimisée ou maximisée, lorsque le solveur calcule les meilleures valeurs des variables de décision

- Les variables de décision : Une variable de décision contient une valeur qui sera ensuite calculée par le solveur pour être la meilleure valeur possible pour minimiser ou maximiser une fonction objectif ( C'est ça, l'optimisation)

- "Sujet à" ou "subject to" veut dire que la fonction objectif est sujette à des contraintes que l'on écrit sous la phrase "subject to".

- ℕ est l'ensemble des entiers positifs y compris le 0, N∗ omets le zéro.

# 2. Notre exemple de base :

On va prendre un modèle assez simple de mixe de production.

Dans les problèmes de débutant, au début, on a pas à utiliser le symbole de sommation, puisqu'on a généralement peu de variables, par déduction, on a pas à itérer avec le solveur.
C'est donc beaucoup plus simple à comprendre, au début.

# L'histoire 

- Une entreprise produit la voiture A et la voiture B.
- La voiture A nécessite 20 unités de fourniture K et 10 unités de fournitures Z.
- La voiture B nécessite 18 unités de fourniture K et 8 unités de fournitures Z.
- Il y a 5000 unités de fournitures K en stock et 6000 unités de fournitures Z en stock.
- La voiture A procure un bénéfice de 20000 euros, et la voiture B procure un bénéfice de 18000 euros.

Quelles voitures doit fournir l'entreprise afin de maximiser son bénéfice, sous contrainte de stocks ?


Donc là, dans ce problème, on voit bien qu'il y a peu de variables de décision en jeu et peu de contraintes.
Aucune raison d'exprimer le modèle mathématique avec des symboles de sommation, et de faire des itérations dans le code du solveur Python.

## On en déduit le modèle mathématique.

Soit

    A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
    B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)

La fonction objectif R est de maximiser le bénéfice :<br>
Max(R) = 20000A + 18000B

Les contraintes sont les contraintes de stock :<br>

Stock de K<br>
20A + 18B <= 5000

Stock de Z<br>
10A + 8B <= 6000

## On écrit le modèle 

Max(R) = 20000A + 18000B<br>
S.T<br>
20A + 18B <= 5000<br>
10A + 8B <= 6000<br>
AVEC<br>
{A,B} ∈ ℕ<br>

## On écrit le code du solveur

In [180]:
# La méthode de base :

# -----------------------------------
# Import de Python Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------

A = LpVariable("A", lowBound=0, cat='Integer')   # Créer une variable x >= 0
B = LpVariable("B", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# -----------------------------------
# Fonction objectif R
# -----------------------------------
model += 20000 * A + 18000  * B

# -----------------------------------
# Contraintes
# -----------------------------------

model += 20 * A + 18  * B <= 5000, "stock_produit_K"
model += 10 * A + 8  * B <= 6000, "stock_produit_Z"


# -----------------------------------
# Solution
# -----------------------------------
model.solve()
 
# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)

# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(model.objective))


A = 7.0
B = 270.0
Profit total maximisé =  5000000.0


## 3. Extention de notre exemple et symbole de sommation.

Admettons désormais que l'entreprise produise 12 voitures différentes et qu'elles nécessitent chacune une quantité de fourniture K et Z différentes, et qu'elles procurent chacune un bénéfice différent. 
L'objectif est toujours de déterminer quelles voitures il vaut mieux produire, afin de maximiser notre bénéfice sous contrainte de stock.
N'est il pas rébarbatif de retaper chaque valeur à la main ? Oui, c'est rébarbatif.

Ne serait-il pas plus pratique d'utiliser le symbole mathématique de sommation lorsque nous allons écrire le modèle mathématique, et d'itérer sur un tableau dans le code du solveur Python? Ceci afin de synthétiser l'expression de notre optimisation ?

Essayons donc désormais de modéliser notre modèle mathématique d'optimisation, avec le symbole de sommation, puis, on va itérer dans le code Python, pour obtenir notre optimisation linéaire finale.


## Tableau des fournitures nécessaires pour les 12 voitures.

In [181]:
# Voitures      A       B       C       D       E       F       G       H       I       J       K       L
# Fourniture K  20      18      9       20      22      10      20      12      15      22      21      5
# Fourniture Z  10      8       8       19      24      10      19      10      12      18      20      4
# Bénéfice      20000   18000   17000   21000   24000   18500   20500   18500   18900   21000   20500   8000 

## On en déduit le modèle mathématique.

Ecrivons cela en dur, afin de comprendre combien c'est rébarbatif.

Soit

    A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
    B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)
    C soit le nombre d'unités de de voitures C produites (C'est une variable de décision)
    D soit le nombre d'unités de de voitures D produites (C'est une variable de décision)
    E soit le nombre d'unités de de voitures E produites (C'est une variable de décision)
    F soit le nombre d'unités de de voitures F produites (C'est une variable de décision)
    G soit le nombre d'unités de de voitures G produites (C'est une variable de décision)
    H soit le nombre d'unités de de voitures H produites (C'est une variable de décision)
    I soit le nombre d'unités de de voitures I produites (C'est une variable de décision)
    J soit le nombre d'unités de de voitures J produites (C'est une variable de décision)
    K soit le nombre d'unités de de voitures K produites (C'est une variable de décision)
    L soit le nombre d'unités de de voitures L produites (C'est une variable de décision)

La fonction objectif R est de maximiser le bénéfice :<br>
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

Les contraintes sont les contraintes de stock :<br>

Stock de K<br>
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

Stock de Z<br>
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

## On écrit le modèle mathématique avec le symbole de sommation et les indices.


On voit bien que ci dessus, on a des sommes, et que les écrire à la main est propice à l'erreur.

- On remarque qu'il existe des ensembles que l'on peut indicer: Les voitures, Les fournitures K, les fournitures Z, les bénéfices.

- On remarque que notre Tableau des fournitures nécessaires pour les 12 voitures est semblable à une matrice Aij.

Rappel :  Soit Une matrice Aij : i c'est le numéro de la ligne, j c'est le numéro de la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4 de la matrice.

- On va donc créer un modèle mathématique, à l'aide d'ensembles, et de symboles de sommations, qui résumera de façon plus simple et plus compréhensible notre problème, ce sera la version synthétisée de notre optimisation.
De plus, ce modèle pourra être utilisé avec tous les solveurs du marché, y compris Excel.


Considérons désormais les ensembles :

- Vj pour les voitures, {A...L} indicé par j
- Kj pour les fournitures K, {20...5} indicé par j
- Zj pour les fournitures Z, {10...4} indicé par j
- Bj pour les bénéfices {20000...8000} indicé par j

On va déjà se servir uniquement des ensembles pour écrire le modèle mathématique, et pas de la matrice; ça, on le fera dans un second temps, car c'est plus compliqué à lire.


Si on veut réécrire la fonction objectif synthétisée qui maximise le bénéfice de la somme des voitures: <br> 
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

On l'écrit : 

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$

- Pourquoi indice-t-on par la lettre j ? Parce que par convention, cela désigne la colonne d'une matrice. Il faut imaginer dans la tête itérer sur chaque valeur et que la valeur de la lettre j s'incrémente.

Si on veut réécrire la première contrainte de stock K, de façon synthétisée :<br> 
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

On l'écrit : 
$$
 \sum_{j=1}^{K} K_{j}.V_{j} <= 5000
$$

Si on veut réécrire la seconde contrainte de stock Z, de façon synthétisée :<br> 
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

On l'écrit : 
$$
 \sum_{j=1}^{Z} Z_{j}.V_{j} <= 6000
$$

On aboutit à notre modèle d'optimisation synthétisé suivant : 

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$
$$
s.t
$$
$$
 \sum_{j=1}^{K} K_{j}.V_{j} <= 5000
$$
$$
 \sum_{j=1}^{Z} Z_{j}.V_{j} <= 6000
$$
$$
 Vj \in ℕ
$$

</b>
<b>
Maintenant, les contraintes ne sont pas écrites comme ça d'habitude, mais font  référence à la matrice :
Source : Programmation linéaire avec Excel de Christian Prins et Marc Sevaux Page 58 et Cours de Taïwan university et Hillier et bien d'autres...
</b>

On peut encore réduire notre modèle en réduisant l'écriture des contraintes de cette façon : 

$$
  \sum_{j=1}^{V} a_{ij}.V_{j} <= S_{i}
$$

Où

- aij veut dire qu'on considère chaque valeur de fourniture pour chaque voiture, pour chaque ligne existante de fourniture dans la matrice de nos données.
- Si veut dire que la valeur de chaque stock actuel est contenu dans un vecteur indicé par i, au lieu d'être renseigné en dur, S est donc l'ensemble des stocks.





## Maintenant, on utilise les solveurs Python 

Reprenons la fonction objectif de notre modèle :

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$

B est donc l'ensemble des bénéfices.
V est donc l'ensemble des autos.

Il va s'agir de récrire ce symbole de sommation, avec Plusieurs Solveurs.

On doit bien sur d'abord entrer les datas de notre tableau initial dans Python, avant de les manipulers avec les Solveurs.
Il existe plusieurs manière d'entrer ces datas, et cela influe ensuite sur la syntaxe lorsqu'on manipule le solveur, néanmoins, on retrouve généralement la même syntaxe, sur StackOverflow et par l'auteur de Python Pulp.
Lire Annexe 18 pour voir différentes méthodes.

In [182]:
# -----------------------------------
# Data
# -----------------------------------
V = ['A','B','C','D','E','F','G','H','I','J','K','L']
B = {'A':20000,'B':18000,'C':17000,'D':21000,'E':24000,'F':18500,'G':20500,'H':18500,'I':18900,'J':21000,'K':20500,'L':8000}

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------

# On stipule que ce sont des variables entières, normal, puisque ce sont des voitures
v = LpVariable.dicts("voitures", V , lowBound=0, cat='Integer')



Allez, c'est parti, on recrée:

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$


- Avec Python Pulp :

In [183]:
# -----------------------------------
# Fonction objectif
# -----------------------------------

model += lpSum([B[j] * v[j] for j in V ]), "MAXIMISER_BENEFICE" 


Avec Cplex :

In [184]:
# A venir

Avec Glpk, en ligne de commande : 

In [185]:
# A venir

# 6. Se familiariser avec l'écriture aij avec un exemple simple tiré de Hillier

## 6.a Présentation du problème dans un tableau

On a vu précédemment que la modélisation des contraintes dans le modèle mathématique utilise souvent aij dans le symbole de sommation.

Prenons ce simple cas d'optimisation de mixe de production, présent dans le livre Hillier, Introduction to operation research p82

L'objectif est de maximiser le profit sous contrainte de temps de production disponible. 



In [186]:
#               Temps de production unité/heure
#                           Produit                 Temps de production disponible 
#               P01     P02     P03     P04 
# Machine       
# Rouleur       1.7     2.1     1.4     2.4         28
# Coupeur       1.1     2.5     1.7     2.6         34  
# Soudure       1.6     1.3     1.6     0.8         21       
# Profit        26      35      25      37

## 6.b Modélisation mathématique

Comme d'habitude, on repère au préalable les ensembles qui font intervenir dans notre modèle mathématique .


Essayons déjà de modéliser <b>la fonction objectif R</b> qui va maximiser le profit:

- On voit qu'on a l'ensemble Xj (x1,x2..xn) qui sont les variables de décision, le j est en indice, parce que on a les 
colonnes P01, P02, P03 et P04. Par convention, généralement, les variables de décision sont appelées x, et on utilise j parce que par convention, on désigne des colonnes.

- On voit qu'on a l'ensemble Pj(p1,p2..pn) qui sont les valeurs constantes de profit par produit fourni. Donc, là c'est pareil, on utilise l'indice j parce que par convention, ça désigne les colonnes.

On aboutit donc à cette fonction objectif, comme dans le livre de Hillier : 


$$
Max (R) = \sum_{j=1}^{4} p_{j}.x_{j}
$$

On remarque qu'on a mis 4 en haut du symbole de sommation, comme limite, mais on aurait aussi pu mettre X, pour obtenir une itération plus générique,  car cela sous entends l'ensemble X qui contient 4 valeurs, mais pourrait en contenir plus.


<b>Maintenant, on en vient à la partie importante, les contraintes .</b>


On voit dans notre tableau qu'on a 3 lignes de contraintes qu'on peut indicer par i. Par convention, i désigne les numéros de lignes.
De ce fait, on ne va pas écrire un symbole de sommation par ligne, ce qui alourdirait notre modèle final, mais on va introduire 
aij qui veut dire qu'on itére automatiquement par ligne et par colonnes en même temps, on modèlise donc nos contraintes ainsi , comme dans le livre de Hillier :

$$
\sum_{j=1}^{4} a_{ij}.x_{j} \thinspace\thinspace\thinspace pour \thinspace i = 1,2,3
$$

On remarque qu'on a une information supplémentaire (obligatoire) " Pour i = 1,2,3 ". Le professeur de Taïwan University insiste particulièrement sur le fait que l'on doit systématiquement ajouter cette information, et elle est présente dans le livre de Hillier.

Voilà, on peut désormais réceecrire notre modèle complet, y compris la nomenclature de nos ensembles, en dessous du modèle.

$$
Max (R) = \sum_{j=1}^{4} p_{j}.x_{j}
$$
$$
S.T
$$

$$
\sum_{j=1}^{4} a_{ij}.x_{j} \thinspace\thinspace\thinspace pour \thinspace i = 1,2,3
$$

$$
Où
$$

$$
 x_{j} = \thinspace Quantité\thinspace  de \thinspace production \thinspace par \thinspace produit \thinspace PO_{j}
$$
$$
 p_{j} = Profit \thinspace unitaire \thinspace par \thinspace produit \thinspace PO_{j}
$$
$$
 a_{ij} = Temps \thinspace de \thinspace production \thinspace par \thinspace machine \thinspace i \thinspace par \thinspace unité \thinspace de \thinspace produit \thinspace PO_{j}
$$
$$
 b_{i} = Temps \thinspace de \thinspace production \thinspace disponible \thinspace par \thinspace semaine\thinspace par\thinspace machine \thinspace i
$$

Note : J'ai recopié le modèle du livre de Hillier, on constate qu'il y a une bizarrerie dans la nomenclature (b est indicé par i dans la nomenclature, et par j dans la contrainte), qui n'est pas exactement la même que dans le modèle, on va vérifier cela sur les autres exemples, mais la logique est là.






## 6.c Solution avec Python Pulp